## AutoLamella-Odemis (Headless)

In [ ]:
# requirements
# stage_position: FibsemStagePosition (stage-bare)                  -> acquire once position selected (can transform from fm-position)
# microcope_state: dict microscope state                            -> acquire once position selected
# reference_image: FibsemImage / OdemisImage (saved to disk)        -> acquire once position selected
# protocol: dict milling protocol for each lamella                  -> load from file, allow user to edit
# config: workflow configuration (supervision, alignment, etc)      -> load from file, allow user to edit

# supported options
# use_microexpansion:

# not supported (for now)
# use_fiducial: # only support non-fiducial milling for now

# odemis: save_milling_position:
# - stage_position:     cryo feature position, defined at milling position: posture_positions[MILLING]
# - microscope_state:   can be extracted from odemis image
# - reference_image:    path to reference image
# - milling_protocol:   dict with milling parameters
#   - patterns:         dict with milling pattern protocol
#   - point:            point in odemis image representing the milling position (generated from correlation)

# req milling parameters
# center -> from correlation / selection
# width  (width of trench)
# height (distance between trenches)
# trench_height (height of trench)
# depth (depth of trench)
# cross_section (cross section of trench Rectangle or CleaningCrossSection)
# milling_current, hfw (all)
# To simplify initially, only use Rough Mill / Polishing

# rough_mill:
# - center: point in odemis image
# - width: 
# - height:
# - trench_height:
# - depth:
# - cross_section: Rectangle

# polishing:
# - center: point in odemis image
# - width:
# - height:
# - trench_height:
# - depth:
# - cross_section: CleaningCrossSection

# microexpansion:
# - center: point in odemis image
# - width:
# - height:
# - distance: distance between expansion trenches
# - depth:

# lamella are created at the ReadyLamella State (after setup in autolamella)
# then the workflow can be run without any user input


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils
from autolamella.waffle import run_autolamella
from autolamella.compat.odemis import add_odemis_path, add_features_to_experiment, create_experiment_from_odemis

import logging
add_odemis_path() # only required when using separate env for openfibsem
from odemis.acq.feature import CryoFeature, model
from odemis import model
from pprint import pprint



In [2]:
# dummy feature creation from odemis
# features = []
# stage_bare = model.getComponent(role="stage-bare")

# while input("Add feature? (y/n): ") == "y":

#     stage_position = stage_bare.position.value
    
#     name = f"Feature-{len(features) + 1}"
#     feature = CryoFeature(name=name, x=stage_position['x'], y=stage_position['y'], z=stage_position['z'])
#     feature.stage_pos = model.VigilantAttribute(stage_position, unit="m")
    
#     features.append(feature)

#     # TODO: acquire FIB REFERENCE IMAGE here

#     logging.info(f"Added feature: {name}, total: {len(features)}")

# dummy feature creation from odemis

stage_positions = [
    {"x": 0, "y": 0, "z": 0, "rx": 0.31, "rz": 0},
    {"x": 50e-6, "y": 50e-6, "z": 0, "rx": 0.31, "rz": 0}
]

features = []
for stage_position in stage_positions:
    name = f"Feature-{len(features) + 1}"
    feature = CryoFeature(name=name, x=stage_position['x'], y=stage_position['y'], z=stage_position['z'])
    feature.stage_pos = model.VigilantAttribute(stage_position, unit="m")
    
    features.append(feature)


logging.info(f"Created {len(features)} features")

In [ ]:
CONFIGURATION_PATH = "/home/patrick/development/openfibsem/fibsem/fibsem/config/odemis-configuration.yaml"
PROTOCOL_PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/protocol/protocol-odemis-on-grid.yaml"
ODEMIS_PROJECT_PATH = "/home/patrick/Pictures/odemis-auto-milling-01"

microscope, settings = utils.setup_session(config_path=CONFIGURATION_PATH, 
                                           protocol_path=PROTOCOL_PATH)

In [ ]:
experiment = create_experiment_from_odemis(ODEMIS_PROJECT_PATH, protocol=settings.protocol)

logging.info(f"Current Experiment: {experiment.name}, total: {len(experiment.positions)}")

# add features to experiment
experiment = add_features_to_experiment(
                                        experiment=experiment, 
                                        features=features,
                                        protocol=settings.protocol
                                        )

logging.info(f"Updated Experiment: {experiment.name}, total: {len(experiment.positions)}")

In [ ]:
# load exp
import os
from autolamella.structures import Experiment
exp = Experiment.load(os.path.join(experiment.path, "experiment.yaml"))

print(exp.positions[0].state.stage)

In [ ]:
from autolamella.structures import AutoLamellaStage
stages_to_complete = [AutoLamellaStage.MillRoughCut]

run_autolamella(microscope=microscope, 
                settings=settings, 
                experiment=experiment,
                parent_ui=None, 
                stages_to_complete=stages_to_complete)

In [ ]:
# TODO: 
# update features after milling